In [2]:
import cv2 as cv
import mediapipe as mp
import pyautogui

# Initialize MediaPipe hands and drawing utilities
mp_hands = mp.solutions.hands
draw = mp.solutions.drawing_utils

def initialize_hands():
    return mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.7, min_tracking_confidence=0.7)

def initialize_camera():
    capture = cv.VideoCapture(0)
    if not capture.isOpened():
        print("Can't open the camera")
        exit()
    return capture

def process_frame(frame, hands):
    vid = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
    return hands.process(vid)

def draw_landmarks(frame, landmark):
    draw.draw_landmarks(frame, landmark, mp_hands.HAND_CONNECTIONS)
    h, w, c = frame.shape
    thumb_tip = landmark.landmark[4]
    index_tip = landmark.landmark[8]
    middle_tip = landmark.landmark[12]
    for id, lm in enumerate(landmark.landmark):
        cx, cy = int(lm.x * w), int(lm.y * h)
        if id in [4, 8, 12]:
            cv.circle(frame, (cx, cy), 15, (255, 255, 0), cv.FILLED)
    return index_tip, middle_tip, thumb_tip, w, h

def move_mouse(index_tip, screen_width, screen_height, w, h):
    screen_x = int(index_tip.x * screen_width)
    screen_y = int(index_tip.y * screen_height)
    pyautogui.moveTo(screen_x, screen_y)

def check_gestures(index_tip, middle_tip, thumb_tip):
    if abs(index_tip.x - thumb_tip.x) < 0.03 and abs(index_tip.y - thumb_tip.y) < 0.03:
        pyautogui.click()
    elif abs(middle_tip.x - thumb_tip.x) < 0.03 and abs(middle_tip.y - thumb_tip.y) < 0.03:
        pyautogui.click(button='right')

def main():
    hands = initialize_hands()
    capture = initialize_camera()
    screen_width, screen_height = pyautogui.size()

    while True:
        ret, frame = capture.read()
        if not ret:
            print("Can't receive frame ...")
            break

        res = process_frame(frame, hands)
        if res.multi_hand_landmarks:
            for landmark in res.multi_hand_landmarks:
                index_tip, middle_tip, thumb_tip, w, h = draw_landmarks(frame, landmark)
                move_mouse(index_tip, screen_width, screen_height, w, h)
                check_gestures(index_tip, middle_tip, thumb_tip)
        
        cv.imshow('Video', frame)
        if cv.waitKey(10) & 0xFF == ord('q'):
            break

    capture.release()
    cv.destroyAllWindows()

if __name__ == "__main__":
    main()
